📰 Google News Scraper (Selenium 기반)

특정 키워드에 대한 구글 뉴스 데이터를 자동으로 수집하고, 타임스탬프를 인간이 읽을 수 있는 날짜 포맷으로 변환하여 저장하는 크롤러입니다.

🚀 주요 기능
- **다중 키워드 지원**: 리스트 형태의 키워드를 순차적으로 검색합니다.
- **날짜 변환**: 구글 뉴스 특유의 `data-ts` 값을 표준 날짜 형식(`YYYY-MM-DD`)으로 자동 변환합니다.
- **중복 방지**: 동일한 URL의 뉴스는 자동으로 필터링합니다.

사용 기술
- Python 3.x
- Selenium
- Pandas
- Webdriver Manager

In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# 구글 뉴스 검색 결과에서 제목, 날짜, 미리보기, 링크를 수집하는 함수
# keywords는 검색할 키워드, target_total은 수집할 데이터 개수
def google_news_scraper(keywords, target_total=500):

    # 브라우저 옵션 설정
    options = Options()
    # 봇 탐지 방지를 위해 실제 브라우저와 유사한 유저 에이전트 설정
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    # 자동화 도구 사용 중임을 나타내는 플래그 제거(차단 방지)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # 웹 드라이버 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    final_results = [] # 수집된 데이터를 담을 리스트
    seen_links = set() # 중복 수집을 방지하기 위한 URL set

    # 키워드별 수집 시작
    for kw in keywords:
        # 최종 리스트의 크기와 목표 수집 개수와 동일해질 때까지 진행
        if len(final_results) >= target_total:
            break
            
        print(f"\n키워드 검색: [{kw}]")
        
        for page in range(0, 50): # 최대 50페이지 탐색
            if len(final_results) >= target_total:
                break
            
            # 구글 검색 URL 파라미터 (10, 20, 30, ...)
            start_index = page * 10
            url = f"https://www.google.com/search?q={kw}&tbm=nws&start={start_index}"
            driver.get(url)
            time.sleep(3.5) # 탐지 방지를 위해 대기

            # 경로의 공통 분모인 a 태그와 내부 컨테이너 위주로 탐색
            articles = driver.find_elements(By.CSS_SELECTOR, 'div#rso > div > div > div')

            if not articles:
                print(f"   - {page+1}페이지에서 뉴스를 찾을 수 없습니다.")
                break

            for article in articles:
                try:
                    # 1) 링크 경로: div > div > a
                    link_el = article.find_element(By.CSS_SELECTOR, 'a')
                    link = link_el.get_attribute('href')
                    
                    # 유효하지 않은 링크나 이미 수집된 링크는 스킵
                    if not link or "google.com" in link or link in seen_links:
                        continue
                    
                    # 2) 제목 경로 (클래스: n0jPhd)
                    try:
                        title = link_el.find_element(By.CSS_SELECTOR, '.n0jPhd').text
                    except:
                        title = "제목 없음"

                    # 3) 미리보기 경로 (클래스: UqSP2b)
                    try:
                        snippet = link_el.find_element(By.CSS_SELECTOR, '.UqSP2b').text
                    except:
                        snippet = ""

                    # 4) 날짜 경로 (span 태그)
                    try:
                        # 보통 날짜는 .OSrXXb 클래스나 특정 span에 위치
                        element = link_el.find_element(By.CSS_SELECTOR, 'span[data-ts]')
                        ts_value = element.get_attribute('data-ts')
                        
                        # 기사 업로드 날짜를 모두 같은 형식으로 통일
                        # 타임스탬프 문자열을 날짜 포맷으로 변환
                        from datetime import datetime

                        def convert_ts_to_date(ts_string):
                            try:
                                # 문자열을 정수로 변환 후 날짜 객체 생성
                                dt_object = datetime.fromtimestamp(int(ts_string))
                                # 원하는 포맷으로 변경 (예: 2026-01-19 14:30:00)
                                return dt_object.strftime('%Y-%m-%d %H:%M:%S')
                            except (ValueError, TypeError):
                                return "날짜 정보 없음"

                        date = convert_ts_to_date(ts_value)

                    except:
                        date = "날짜 미상"

                    # 수집된 결과물을 리스트에 추가
                    final_results.append({
                        "키워드": kw,
                        "제목": title.strip(),
                        "날짜": date.strip(),
                        "미리보기": snippet.strip(),
                        "링크": link
                    })
                    seen_links.add(link) # 중복 확인을 위해 set에 추가

                    # 목표 개수 달성 시 break
                    if len(final_results) >= target_total:
                        break
                except:
                    continue
            
            print(f"   - 현재 수집량: {len(final_results)}개")

    # 모든 작업 종료 후 브라우저 닫기
    driver.quit()
    return final_results

# --- 실행 및 CSV 저장 ---
search_list = ["철강 전기", "산업 전기", "전력 효율", "공장 전기"]
data = google_news_scraper(search_list, 500)

if data:
    df = pd.DataFrame(data)
    # 중복 제거 (혹시 모를 중복 방지)
    df = df.drop_duplicates(subset=['링크'])
    # csv파일로 저장
    df.to_csv("google_news_withdate.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ 완료! 'google_news_withdate.csv'에 {len(df)}건 저장됨")
else:
    print("\n❌ 수집된 데이터가 없습니다. 셀렉터를 다시 점검해야 합니다.")


🚀 키워드 검색: [철강 전기]
   - 현재 수집량: 10개
   - 현재 수집량: 20개
   - 현재 수집량: 30개
   - 현재 수집량: 40개
   - 현재 수집량: 50개
   - 현재 수집량: 60개
   - 현재 수집량: 70개
   - 현재 수집량: 80개
   - 현재 수집량: 90개
   - 현재 수집량: 100개
   - 현재 수집량: 110개
   - 현재 수집량: 120개
   - 현재 수집량: 130개
   - 현재 수집량: 140개
   - 현재 수집량: 150개
   - 현재 수집량: 160개
   - 현재 수집량: 170개
   - 현재 수집량: 180개
   - 현재 수집량: 190개
   - 현재 수집량: 200개
   - 현재 수집량: 210개
   - 현재 수집량: 220개
   - 현재 수집량: 230개
   - 현재 수집량: 240개
   - 현재 수집량: 250개
   - 현재 수집량: 260개
   - 현재 수집량: 270개
   - 현재 수집량: 280개
   - 현재 수집량: 290개
   - 현재 수집량: 300개
   - 현재 수집량: 310개
   - 현재 수집량: 320개
   - 현재 수집량: 330개
   - 현재 수집량: 340개
   - 현재 수집량: 350개
   - 현재 수집량: 352개
   - 37페이지에서 뉴스를 찾을 수 없습니다.

🚀 키워드 검색: [산업 전기]
   - 현재 수집량: 362개
   - 현재 수집량: 372개
   - 현재 수집량: 380개
   - 현재 수집량: 390개
   - 현재 수집량: 400개
   - 현재 수집량: 410개
   - 현재 수집량: 418개
   - 현재 수집량: 428개
   - 현재 수집량: 437개
   - 현재 수집량: 446개
   - 현재 수집량: 456개
   - 현재 수집량: 465개
   - 현재 수집량: 474개
   - 현재 수집량: 483개
   - 현재 수집량: 493개
   - 현재 수집량: 500개

✅ 완료! 

💡 코드의 핵심 포인트
1. 봇 탐지 회피 전략
- User-Agent 설정: 실제 사용자가 크롬을 사용하는 것처럼 속입니다.
- Automation Flag 제거: 셀레니움 특유의 자동화 표시를 지웁니다.
- Time Sleep: 요청 사이에 넉넉한 간격(3.5초)을 두어 서버 부하를 줄이고 차단을 피합니다.

2. 데이터의 정확성
- Timestamp 변환: 구글 검색 결과의 유동적인 "3시간 전", "1일 전" 표기 대신 data-ts 속성을 추출하여 정확한 절대 날짜로 변환합니다.
- 중복 제거: seen_links라는 set 자료형을 사용하여 중복 수집을 원천 차단하고 메모리 효율을 높였습니다.

3. 안정성 (Exception Handling)
- 기사 하나를 읽는 데 실패하더라도 전체 프로그램이 멈추지 않도록 각 단계마다 try-except 구문을 배치하여 연속적인 크롤링이 가능하도록 설계되었습니다.